Pandas and duckdb

###  Task 0

In [ ]:
# Task 0

import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

        

films.head()  

In [ ]:
description.head()

### Task 1

###  Read all data into dictionary of pandas dataframes

In [ ]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()

dfs.keys()

In [ ]:
# task 1a)

length_of_movie=duckdb.sql("""    
    SELECT title, length AS movie_length
    FROM films
    WHERE length > 180;         
    """).df()

length_of_movie.head()

In [ ]:
# task 1b)

love_word=duckdb.sql("""
SELECT title, rating, length, description
FROM films
WHERE title ILIKE '%love%';
""").df()

love_word.head()

In [ ]:
# task 1c)

length_statistik=duckdb.sql("""
SELECT
MIN(length) AS shortest,
MAX(length) AS longest,
AVG(length) AS average,
MEDIAN(length) AS median 
FROM films;                          
""").df()

length_statistik.head()



In [ ]:
# task 1d)